In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import TruncatedSVD

In [2]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
#Drop missing values of embarked
df = df.loc[df['Embarked'].notnull()]
len(df)

889

In [5]:
X = df.drop(['Survived', 'PassengerId', 'Cabin', 'Ticket'], axis=1) #Drop cabin for simplicity
y = df.Survived
print(X.shape, y.shape)

(889, 8) (889,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

(711, 8) (711,)


In [7]:
imputer = SimpleImputer()
scaler = StandardScaler()
one_hot = OneHotEncoder(handle_unknown = 'ignore')
vectorizer = TfidfVectorizer()
lr = LogisticRegression()

In [8]:
numeric_transformer = make_pipeline((imputer),
                                   (scaler))
categorical_transformer = make_pipeline(one_hot)
svd = TruncatedSVD(n_components=2, random_state = 42)
text_transformer = make_pipeline((vectorizer), (svd))

In [9]:
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
cat_features = ['Sex', 'Embarked']
text_features = 'Name'

In [10]:
col_transformer = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, cat_features),
    (text_transformer, 'Name'),  
    remainder='passthrough',
)

In [11]:
X_train.iloc[0]

Pclass                         1
Name        Cleaver, Miss. Alice
Sex                       female
Age                        22.00
SibSp                          0
Parch                          0
Fare                      151.55
Embarked                       S
Name: 708, dtype: object

In [12]:
x = col_transformer.fit_transform(X_train)
x[0]

array([-0.60290568, -0.47451561, -0.4756437 ,  2.43059683,  1.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.0193423 ,
        0.31031523,  1.        ])

In [13]:
pipe = make_pipeline(col_transformer,SelectKBest(f_regression), lr)

In [78]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'selectkbest', 'logisticregression', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__pipeline-1', 'columntransformer__pipeline-2', 'columntransformer__pipeline-3', 'columntransformer__pipeline-1__memory', 'columntransformer__pipeline-1__steps', 'columntransformer__pipeline-1__verbose', 'columntransformer__pipeline-1__simpleimputer', 'columntransformer__pipeline-1__standardscaler', 'columntransformer__pipeline-1__simpleimputer__add_indicator', 'columntransformer__pipeline-1__simpleimputer__copy', 'columntransformer__pipeline-1__simpleimputer__fill_value', 'columntransformer__pipeline-1__simpleimputer__missing_values', 'columntransformer__pipeline-1__simpleimputer__strategy', 'columntransformer__pipeline-1__simpleimputer__verbose', 'columntransformer_

In [14]:
params = {'selectkbest__k' : [7,8],
         'logisticregression__C' : [0.01, 1],
         'columntransformer__pipeline-3__truncatedsvd__n_components' : [2,3]}

grid_search = GridSearchCV(pipe, param_grid=params, cv=3, scoring='accuracy', refit=True)

In [15]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                     

In [16]:
grid_search.best_estimator_

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked']),
                             

In [17]:
grid_search.best_score_

0.7862165963431785

In [18]:
grid_search.best_params_

{'columntransformer__pipeline-3__truncatedsvd__n_components': 3,
 'logisticregression__C': 1,
 'selectkbest__k': 7}

In [19]:
y_train.value_counts(normalize=True)

0    0.618847
1    0.381153
Name: Survived, dtype: float64

In [44]:
X_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
281,3,"Olsson, Mr. Nils Johan Goransson",male,28.0,0,0,7.8542,S
435,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,120.0000,S
39,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,11.2417,C
418,2,"Matthews, Mr. William John",male,30.0,0,0,13.0000,S
585,1,"Taussig, Miss. Ruth",female,18.0,0,2,79.6500,S


In [52]:
oos = X_test.head()
y_test[:5]

281    0
435    1
39     1
418    0
585    1
Name: Survived, dtype: int64

In [49]:
grid_search.predict(oos)

array([0, 1], dtype=int64)